# Tutorial 03: Memory & Persistence

In this tutorial, you'll learn how to add memory to your LangGraph agents so they can remember previous conversations.

**What you'll learn:**
- **Checkpointers**: How LangGraph persists state
- **Thread IDs**: Maintaining separate conversations
- **MemorySaver**: In-memory persistence for development
- **SqliteSaver**: File-based persistence for production
- **State inspection**: Viewing conversation history

By the end, you'll have a chatbot that remembers your conversation across multiple interactions.

## The Memory Problem

In Tutorial 01, we built a basic chatbot. But it had a critical limitation:

```python
# Each call is independent - no memory!
result1 = graph.invoke({"messages": [("user", "My name is Alice")]})
result2 = graph.invoke({"messages": [("user", "What's my name?")]})  # Doesn't know!
```

Each `invoke()` starts fresh. The graph has no way to remember what happened before.

### The Solution: Checkpointers

LangGraph solves this with **checkpointers** - components that save the graph state after each step. When you compile a graph with a checkpointer:

1. After each node runs, state is saved to the checkpointer
2. State is organized by **thread ID** (like a conversation ID)
3. Future calls with the same thread ID continue from the saved state

## Graph Visualization

The graph structure is the same as Tutorial 01, but now with persistence:

![Memory Graph](../docs/images/03-memory-graph.png)

The key difference is in how we **compile** the graph - we add a checkpointer.

In [ ]:
# Setup: Verify connection
from langgraph_ollama_local import LocalAgentConfig

config = LocalAgentConfig()
print(f"Ollama: {config.ollama.base_url}")
print(f"Model: {config.ollama.model}")

## Step 1: Build a Basic Chatbot (No Memory)

First, let's recreate our chatbot from Tutorial 01 to demonstrate the problem:

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_ollama import ChatOllama

# State
class State(TypedDict):
    messages: Annotated[list, add_messages]

# LLM
llm = ChatOllama(
    model=config.ollama.model,
    base_url=config.ollama.base_url,
)

# Node
def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}

# Graph WITHOUT checkpointer
graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

# Compile without checkpointer
graph_no_memory = graph_builder.compile()

print("Graph compiled (no memory)")

In [ ]:
# Demonstrate the problem: no memory between calls
result1 = graph_no_memory.invoke({"messages": [("user", "My name is Alice.")]})
print(f"Response 1: {result1['messages'][-1].content}")
print()

result2 = graph_no_memory.invoke({"messages": [("user", "What is my name?")]})
print(f"Response 2: {result2['messages'][-1].content}")
print()
print("Note: The chatbot doesn't remember your name!")

## Step 2: Add MemorySaver

Now let's add memory using `MemorySaver`, the simplest checkpointer.

`MemorySaver` stores state in memory (Python dictionaries). It's perfect for:
- Development and testing
- Single-session applications
- Learning LangGraph concepts

**Limitation**: State is lost when the Python process ends.

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

# Create a MemorySaver
memory = MemorySaver()

# Compile the SAME graph with checkpointer
graph_with_memory = graph_builder.compile(checkpointer=memory)

print("Graph compiled with MemorySaver!")

## Step 3: Using Thread IDs

When using a checkpointer, you must provide a **thread ID** in the config. This is like a conversation ID that groups related messages.

```python
config = {"configurable": {"thread_id": "my-conversation-1"}}
result = graph.invoke(input, config=config)
```

In [ ]:
# Create a thread config
thread_config = {"configurable": {"thread_id": "conversation-1"}}

# First message
result1 = graph_with_memory.invoke(
    {"messages": [("user", "My name is Alice.")]},
    config=thread_config
)
print(f"Response 1: {result1['messages'][-1].content}")
print()

In [ ]:
# Second message - same thread
result2 = graph_with_memory.invoke(
    {"messages": [("user", "What is my name?")]},
    config=thread_config  # Same thread ID!
)
print(f"Response 2: {result2['messages'][-1].content}")
print()
print("The chatbot remembers your name!")

## Step 4: Inspecting State

You can inspect the saved state for any thread using `get_state()`:

In [ ]:
# Get the current state of our thread
state = graph_with_memory.get_state(thread_config)

print("Current thread state:")
print(f"  Thread ID: {thread_config['configurable']['thread_id']}")
print(f"  Number of messages: {len(state.values['messages'])}")
print()
print("Messages:")
for i, msg in enumerate(state.values['messages']):
    role = msg.type if hasattr(msg, 'type') else 'unknown'
    content = msg.content[:100] + '...' if len(msg.content) > 100 else msg.content
    print(f"  {i+1}. [{role}]: {content}")

## Step 5: Multiple Threads

Different thread IDs maintain separate conversation histories:

In [ ]:
# Thread 2: A different conversation
thread2_config = {"configurable": {"thread_id": "conversation-2"}}

result = graph_with_memory.invoke(
    {"messages": [("user", "My name is Bob.")]},
    config=thread2_config
)
print(f"Thread 2 Response: {result['messages'][-1].content}")

In [ ]:
# Check: Thread 1 still remembers Alice
result = graph_with_memory.invoke(
    {"messages": [("user", "Remind me, what's my name?")]},
    config=thread_config  # Back to thread 1
)
print(f"Thread 1 Response: {result['messages'][-1].content}")
print()
print("Each thread maintains its own conversation history!")

## Step 6: SqliteSaver for Persistent Storage

`MemorySaver` is great for development, but data is lost when the process ends.

For persistent storage, use `SqliteSaver` which stores checkpoints in a SQLite database file.

**Note**: SqliteSaver requires the `langgraph-checkpoint-sqlite` package:
```bash
pip install langgraph-checkpoint-sqlite
```

In [ ]:
import sqlite3
from langgraph.checkpoint.sqlite import SqliteSaver

# Create SQLite connection
# Use :memory: for in-memory SQLite (good for demos)
# Or use a file path like "checkpoints.db" for persistence
conn = sqlite3.connect(":memory:", check_same_thread=False)

# Create SqliteSaver
sqlite_memory = SqliteSaver(conn)

# Compile graph with SqliteSaver
graph_sqlite = graph_builder.compile(checkpointer=sqlite_memory)

print("Graph compiled with SqliteSaver!")

In [ ]:
# Test SqliteSaver
sqlite_config = {"configurable": {"thread_id": "sqlite-thread-1"}}

result = graph_sqlite.invoke(
    {"messages": [("user", "Remember this: The secret code is 42.")]},
    config=sqlite_config
)
print(f"Response: {result['messages'][-1].content}")

In [ ]:
# Verify it remembers
result = graph_sqlite.invoke(
    {"messages": [("user", "What was the secret code?")]},
    config=sqlite_config
)
print(f"Response: {result['messages'][-1].content}")

## Step 7: File-Based Persistence

For true persistence across Python sessions, use a file-based SQLite database:

In [ ]:
from pathlib import Path

# Create a persistent database file
db_path = Path(".checkpoints/conversations.db")
db_path.parent.mkdir(exist_ok=True)

# Connect to file-based SQLite
persistent_conn = sqlite3.connect(str(db_path), check_same_thread=False)
persistent_memory = SqliteSaver(persistent_conn)

# Compile graph
graph_persistent = graph_builder.compile(checkpointer=persistent_memory)

print(f"Database file: {db_path.absolute()}")
print("This database persists across Python sessions!")

## Complete Code: Chatbot with Memory

Here's the complete implementation:

In [ ]:
# Complete Chatbot with Memory

from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langchain_ollama import ChatOllama
from langgraph_ollama_local import LocalAgentConfig

# === State ===
class State(TypedDict):
    messages: Annotated[list, add_messages]

# === LLM ===
config = LocalAgentConfig()
llm = ChatOllama(
    model=config.ollama.model,
    base_url=config.ollama.base_url,
)

# === Node ===
def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}

# === Graph ===
graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

# === Compile with Checkpointer ===
memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

# === Use It ===
def chat(user_input: str, thread_id: str = "default"):
    """Send a message and get a response."""
    config = {"configurable": {"thread_id": thread_id}}
    result = graph.invoke({"messages": [("user", user_input)]}, config=config)
    return result["messages"][-1].content

# Test it
print(chat("Hi! I'm learning LangGraph."))
print()
print(chat("What am I learning?"))  # Remembers!

## Key Concepts Recap

| Concept | Description |
|---------|-------------|
| **Checkpointer** | Component that saves graph state after each step |
| **Thread ID** | Identifier for a conversation (groups related messages) |
| **MemorySaver** | In-memory checkpointer (lost on restart) |
| **SqliteSaver** | SQLite-based checkpointer (persistent) |
| **get_state()** | Inspect the current state of a thread |

## Checkpointer Comparison

| Checkpointer | Persistence | Use Case |
|--------------|-------------|----------|
| `MemorySaver` | In-memory only | Development, testing |
| `SqliteSaver` | File-based | Local applications, demos |
| `PostgresSaver` | Database | Production, multi-server |

## What's Next?

In [Tutorial 04: Human-in-the-Loop](04_human_in_the_loop.ipynb), you'll learn:
- Pausing agent execution for human approval
- Using `interrupt_before` and `interrupt_after`
- Reviewing and modifying agent actions before they execute